## 1. Environment Setup

`(1) Env Environment Variables`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) Basic Libraries`

In [2]:
import re
import os, json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

## 2. State Reducer
- A Reducer is an important concept in LangGraph for managing state updates.
- It defines how to integrate the output of each node in the graph into the graph's state.
- Necessity of a Reducer
    - State Overwriting Problem: By default, the return value of each node overwrites the previous value of the corresponding state key (override).
    - Need for Cumulative Updates: Especially in cases like a list of messages, you may want to add new values to the previous state.

`(1) When a Reducer is not specified separately `
- If a reducer is not specified separately, it operates by overwriting the existing value.
- The default reducer is automatically applied when used without any separate configuration for the state.

In [3]:
from typing import TypedDict, List, Annotated

# Custom reducer: merge lists while removing duplicate documents
def reduce_unique_documents(left: list | None, right: list | None) -> list:
    """Combine two lists of documents, removing duplicates."""
    if not left:
        left = []
    if not right:
        right = []
    # Remove duplicates: use a set to remove duplicate documents and then convert back to a list
    return list(set(left + right))

# Define State (including documents field)
class CustomReducerState(TypedDict):
    query: str
    documents: Annotated[List[str], reduce_unique_documents]  # Apply Custom Reducer


# Node 1: query update
def node_1(state: CustomReducerState) -> CustomReducerState:
    print("---Node 1 (query update)---")
    query = state["query"]
    return {"query": query}

# Node 2: add documents
def node_2(state: CustomReducerState) -> CustomReducerState:
    print("---Node 2 (add documents)---")
    return {"documents": ["doc1.pdf", "doc2.pdf", "doc3.pdf"]}

# Node 3: add more documents
def node_3(state: CustomReducerState) -> CustomReducerState:
    print("---Node 3 (add more documents)---")
    return {"documents": ["doc2.pdf", "doc4.pdf", "doc5.pdf"]}

# Build Graph
builder = StateGraph(CustomReducerState)
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_node("node_3", node_3)

# Configure Logic
builder.add_edge(START, "node_1")
builder.add_edge("node_1", "node_2")
builder.add_edge("node_2", "node_3")
builder.add_edge("node_3", END)

# Execute Graph
graph = builder.compile()

# Visualize Graph
display(Image(graph.get_graph().draw_mermaid_png()))

In [8]:
# Initial State
initial_state = {"query": "Recommend a vegan dish for a vegetarian.", "documents": []}

# Execute Graph
final_state = graph.invoke(initial_state)

# Print Final State
print("Final State:", final_state)

---Node 1 (query update)---
---Node 2 (add documents)---
---Node 3 (add more documents)---
Final State: {'query': 'Recommend a vegan dish for a vegetarian.', 'documents': ['doc1.pdf', 'doc4.pdf', 'doc3.pdf', 'doc2.pdf', 'doc5.pdf']}


## 3. MessageGraph
- LangChain's ChatModel processes a list of Message objects as input (a special type of StateGraph).
- These messages are provided in various forms, such as HumanMessage (user input) or AIMessage (LLM response).

`(1) Defining Messages State`
- It is useful to save the previous conversation history as a list of messages in the graph state.
- Add a key (channel) to the graph state to store the list of Message objects, and add a reducer function to this key.
- Reducer function selection:
    - Using operator.add: simply adds new messages to the existing list.
    - Using the add_messages function:
        - New messages are added to the existing list.
        - Updates to existing messages are also handled correctly (by tracking message IDs).

In [9]:
from typing import Annotated
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages

# Use the basic State initialization method
class GraphState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [10]:
# Use the pre-built state called LangGraph MessagesState
from langgraph.graph import MessagesState
from typing import List
from langchain_core.documents import Document

class GraphState(MessagesState):
    # The 'messages' key is provided by default - if you want to add other keys, you can apply them as shown in the comments below
    documents: List[Document]
    grade: float
    num_generation: int

`(2) RAG Chain Configuration`
- Initialize the vector store for menu search (load the existing store).
- Implemented as a LangChain Runnable.

In [11]:
from langchain_chroma import Chroma
from langchain_ollama  import OllamaEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

embeddings_model = OllamaEmbeddings(model="bge-m3") 

# Load Chroma index
vector_db = Chroma(
    embedding_function=embeddings_model,   
    collection_name="restaurant_menu",
    persist_directory="./chroma_db",
)

# LLM Model
llm = ChatOpenAI(model="gpt-4o-mini")

# Configure RAG chain
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

system = """
You are a helpful assistant. Use the following context to answer the user's question:

[Context]
{context}
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    ("human", "{question}")
])

# Define retriever
retriever = vector_db.as_retriever(
    search_kwargs={"k": 2}
)

# Configure RAG chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Execute RAG chain
query = "Recommend a menu for a vegetarian."
response = rag_chain.invoke(query)

# Print response
print(response)

We recommend the Garden Salad for vegetarians.

The Garden Salad consists of fresh vegetables such as organic mixed greens, cherry tomatoes, cucumbers, and carrots, and the healthy balsamic dressing enhances the taste of the vegetables. The price is ₩12,000. It is a suitable menu for vegetarians as you can enjoy various tastes and crispy textures.


`(3) Node`

In [15]:
# Define RAG execution function
def retrieve_and_respond(state: GraphState):
    last_human_message = state['messages'][-1]
    
    # Access the content attribute of the HumanMessage object
    query = last_human_message.content
    
    # Document retrieval
    retrieved_docs = retriever.invoke(query)
    
    # Generate response
    response = rag_chain.invoke(query)
    
    # Save the retrieved documents and response to the state
    return {
        "messages": [AIMessage(content=response)],
        "documents": retrieved_docs
    }

In [12]:
from pydantic import BaseModel, Field

class GradeResponse(BaseModel):
    "A score for answers"
    score: float = Field(..., ge=0, le=1, description="A score from 0 to 1, where 1 is perfect")
    explanation: str = Field(..., description="An explanation for the given score")

# Answer quality evaluation function
def grade_answer(state: GraphState):
    messages = state['messages']
    question = messages[-2].content
    answer = messages[-1].content
    context = format_docs(state['documents'])

    grading_system = """You are an expert grader. 
    Grade the following answer based on its relevance and accuracy to the question, considering the given context. 
    Provide a score from 0 to 1, where 1 is perfect, along with an explanation."""

    grading_prompt = ChatPromptTemplate.from_messages([
        ("system", grading_system),
        ("human", "[Question]\n{question}\n\n[Context]\n{context}\n\n[Answer]\n{answer}\n\n[Grade]\n")
    ])
    
    grading_chain = grading_prompt | llm.with_structured_output(schema=GradeResponse)
    
    grade_response = grading_chain.invoke({
        "question": question,
        "context": context,
        "answer": answer
    })

    # Increment the number of response generations
    num_generation = state.get('num_generation', 0)
    num_generation += 1
    
    return {"grade": grade_response.score, "num_generation": num_generation}

`(4) Edge`

In [13]:
from typing import Literal


def should_retry(state: GraphState) -> Literal["retrieve_and_respond", "generate"]:
    print("----GRADTING---")
    print("Grade Score: ", state["grade"])

    # If the number of response generations is 3 or more, return "generate"
    if state["num_generation"] > 2: 
        return "generate"    
    
    # If the answer quality score is less than 0.7, re-run the RAG chain
    if state["grade"] < 0.7:  
        return "retrieve_and_respond"
    else:
        return "generate"

`(5) Graph Configuration`

In [16]:
# Graph setup
builder = StateGraph(GraphState)
builder.add_node("retrieve_and_respond", retrieve_and_respond)
builder.add_node("grade_answer", grade_answer)

builder.add_edge(START, "retrieve_and_respond")
builder.add_edge("retrieve_and_respond", "grade_answer")
builder.add_conditional_edges(
    "grade_answer",
    should_retry,
    {
        "retrieve_and_respond": "retrieve_and_respond",
        "generate": END
    }
)

# Compile graph
graph = builder.compile()

# Visualize graph
display(Image(graph.get_graph().draw_mermaid_png()))

`(6) Execute Graph`

In [17]:
# Initial state
initial_state = {
    "messages": [HumanMessage(content="Recommend a menu for a vegetarian.")],
}

# Execute graph
final_state = graph.invoke(initial_state)

# Print final state
print("Final State:", final_state)

----GRADTING---
Grade Score:  1.0
Final State: {'messages': [HumanMessage(content='Recommend a menu for a vegetarian.', additional_kwargs={}, response_metadata={}, id='2aee9ce3-7511-422a-913c-805aca651005'), AIMessage(content="For vegetarians, we recommend the 'Garden Salad'. This salad is composed of fresh vegetables such as organic mixed greens, cherry tomatoes, cucumbers, and carrots, and the balsamic dressing enhances the taste of the vegetables. It is a menu where you can enjoy a healthy and crispy texture. The price is ₩12,000.", additional_kwargs={}, response_metadata={}, id='1c1cf75b-26b9-4879-9108-64135b3c3535')], 'documents': [Document(metadata={'menu_name': 'Garden Salad', 'menu_number': 5, 'source': './data/restaurant_menu.txt'}, page_content='5. Garden Salad\n   • Price: ₩12,000\n   • Main Ingredients: Organic mixed greens, cherry tomatoes, cucumbers, carrots, balsamic dressing\n   • Description: A healthy salad composed of fresh organic vegetables. You can enjoy various t

In [18]:
# Print only the final answer
pprint(final_state['messages'][-1].content) 

("For vegetarians, we recommend the 'Garden Salad'. This salad is composed of fresh vegetables such as organic mixed greens, cherry tomatoes, cucumbers, and carrots, and the balsamic dressing enhances the taste of the vegetables. It is a menu where you can enjoy a healthy and crispy texture. The price is ₩12,000.")


## 4. Gradio Chatbot

In [19]:
import gradio as gr
from typing import List, Tuple

# Example questions
example_questions = [
    "Recommend a menu for a vegetarian.",
    "What is today's special menu?",
    "What drink goes well with pasta?"
]

# Define answer function
def answer_invoke(message: str, history: List[Tuple[str, str]]) -> str:
    try:
        # Convert chat history to a format that can be passed to the AI
        chat_history = []
        for human, ai in history:
            chat_history.append(HumanMessage(content=human))
            chat_history.append(AIMessage(content=ai))

        # Add the user's message to the existing chat history (use only the last 2 conversations)
        initial_state = {
            "messages": chat_history[-2:]+[HumanMessage(content=message)],  
        }

        # Process the message and return the final state
        final_state = graph.invoke(initial_state)
        
        # Return the necessary part from the final state (e.g., recommended menu, etc.)
        return final_state["messages"][-1].content
        
    except Exception as e:
        # Notify the user and log the error if one occurs
        print(f"Error occurred: {str(e)}")
        return "Sorry, an error occurred while generating a response. Please try again."


# Create Gradio interface
demo = gr.ChatInterface(
    fn=answer_invoke,
    title="Restaurant Menu AI Assistant",
    description="We will answer your questions about menu information, recommendations, and food.",
    examples=example_questions,
    theme=gr.themes.Soft()
)

# Launch Gradio app
demo.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


----GRADTING---
Grade Score:  1.0
----GRADTING---
Grade Score:  1.0
----GRADTING---
Grade Score:  1.0
----GRADTING---
Grade Score:  1.0


In [20]:
# Close demo
demo.close()


Closing server running on port: 7860
